## I want to extract the features from other files B to F from .txt file using NLP approach

In [7]:
!pip install tensorflow

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


In [16]:
import sys
print(sys.executable)
print(sys.version)

e:\python.exe
3.13.2 (tags/v3.13.2:4f8bb39, Feb  4 2025, 15:23:48) [MSC v.1942 64 bit (AMD64)]


In [1]:
!python --version

Python 3.13.2


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from transformers import pipeline

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
# sentiment analysis model iniaalization
sentiment_analyzer = pipeline("sentiment-analysis")

# Function to extract features from video scripts
def extract_features(script):
    sentiment = sentiment_analyzer(script)[0]
    return {
        'sentiment_label': sentiment['label'],
        'sentiment_score': sentiment['score'],
        'length': len(script.split())
    }

# Load scripts and extract features
video_scripts = {}
features = []

for video in ['A', 'B', 'C', 'D', 'E', 'F']:
    with open(f'Video_{video}.txt', 'r') as file:
        script = file.read()
        features.append({**extract_features(script), 'video': video})

features_df = pd.DataFrame(features)
features_df.head()

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [ ]:
# Audience retention data from B to F
retention_data = {}
for video in ['B', 'C', 'D', 'E', 'F']:
    retention_data[video] = pd.read_excel(f'Video_{video}.xlsx', sheet_name='Organic')

# features with retention data combinations
for video in ['B', 'C', 'D', 'E', 'F']:
    features_df = features_df.merge(retention_data[video], on='video', how='left')

In [ ]:
## Modeling

# Prepare features and target
X = features_df[['sentiment_score', 'length']]  # Add other relevant features
y = features_df['absolute_audience_retention']  # Assuming this is the column name

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Evaluate performance
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

In [ ]:
# predict the missing part of absolute retention in Videa A.xslx
# Prepare input for Video A
video_a_features = extract_features(open('Video_A.txt').read())
video_a_df = pd.DataFrame([video_a_features])

# Predict missing values
predicted_values = model.predict(video_a_df[['sentiment_score', 'length']])
print(f'Predicted Audience Retention for Video A: {predicted_values}')